In [4]:
from datasharing.datasharing import DataSharingClient
import pandas as pd
import matplotlib.pyplot as plt
import re
import polars as pl

In [5]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pl.Config.set_tbl_rows(100) 
pl.Config.set_tbl_cols(10)

polars.config.Config

In [9]:
client = DataSharingClient(debug=True)

local_path = 'C:/Users/lazzz/PycharmProjects/datasharing/data_used/francetax.parquet'
view_name = "francetax"
client.create_view(local_path, view_name)

Authentication successful.
Getting identity ID...
Identity ID obtained: us-east-1:965f31bc-4398-ceed-80d7-4c23051bccb6
Getting OpenID token...
OpenID token obtained.
Getting credentials for identity...
Temporary credentials obtained.
DuckDB setup complete.
View francetax created.


# Fix missing INSEE codes

# Data preparation

## Which columns to use

In [10]:
query="""SELECT column_name
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'francetax';
"""
column_names = client.query(query)

Query: SELECT column_name
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'francetax';



In [11]:
# DO NOT CHANGE ORDER OF TAXES

tax_types = [
    "FNB",
    "TAFNB",
    "FB",
    "TH",
    "CFE",
    "CVAE", 
]
y_columns = [
    "FNB - COMMUNE / MONTANT REEL",
    "TAFNB - COMMUNE / MONTANT REEL NET",
    "FB - COMMUNE / MONTANT REEL",
    "TH - COMMUNE / MONTANT REEL DONT THP/E AU PROFIT DE L ETAT",
    "CFE - COMMUNE / PRODUIT REEL NET",
    "Part de CVAE au profit de la commune", 
]

In [12]:
dict_col_prediction = {}
for tax in tax_types:
    # find columns, which contain tax type in their name, return as list to dict
    matching_columns = column_names[column_names['column_name'].str.contains(r'\b' + re.escape(tax) + r'\b', regex=True, case=False)]['column_name'].tolist()
    # Exclude columns that are in y_columns
    # matching_columns = [col for col in matching_columns if col not in y_columns]
    dict_col_prediction[tax] = matching_columns

## EXCLUDE TEOM, IFER, TASCOM because target columns are almos empty

## Collecting data

In [7]:
# alldata_pl = pl.read_parquet('data_used/francetax.parquet')

In [8]:
# francetax = pl.scan_parquet('data_used/francetax.parquet')

In [13]:
francetax = pl.scan_parquet('data_used/francetax.parquet').with_columns(
    # Replace - ' in commune names
    pl.col("Libellé commune").str.replace("-", " ").str.replace("'", "").alias("Commune name"),
    # Fix DEPARTEMENT column => replace \n symbols and 0, ["96", "98", "99"] == 97*
    pl.when(pl.col("DEPARTEMENT").cast(pl.Utf8).str.replace("\n", "").str.replace(r"^0+", "").is_in(["96", "98", "99"]))
    .then(pl.concat_str([pl.lit("97"), pl.col("DEPARTEMENT").cast(pl.Utf8).str.replace("\n", "").str.replace(r"^0+", "").str.slice(-1)]))
    .otherwise(pl.col("DEPARTEMENT").cast(pl.Utf8).str.replace("\n", "").str.replace(r"^0+", "")).alias("DEPARTEMENT"),
    # Fix COMMUNE column
    pl.when((pl.col("DEPARTEMENT").cast(pl.Utf8).str.replace("\n", "").str.replace(r"^0+", "").is_in(["96", "98", "99"])) & (pl.col("COMMUNE").cast(pl.Utf8).str.len_chars() == 3))
    .then(pl.col("COMMUNE").cast(pl.Utf8).str.slice(-2))
    .when(pl.col("COMMUNE") == 999)
    .then(pl.lit("56"))
    .otherwise(pl.col("COMMUNE").cast(pl.Utf8).str.replace("'", ""))
    .alias("COMMUNE")
)
basic_columns = ['Commune name', 'DEPARTEMENT', 'COMMUNE']

In [14]:
def create_insee_code(department, commune):
    # Define conditions and actions for creating the INSEE code
    condition = department.str.len_chars() <= 2
    action1 = department.str.zfill(2) + commune.str.zfill(3)
    action2 = department + commune.str.zfill(2)
    # Use a ternary expression to apply the appropriate action
    insee_code = pl.when(condition).then(action1).otherwise(action2)
    return insee_code

In [15]:
commune_code = francetax.select(*basic_columns).collect()

commune_code = commune_code.with_columns([
    create_insee_code(commune_code['DEPARTEMENT'], commune_code['COMMUNE']).alias("code INSEE")
])

## Query and clean the data

In [12]:
pred_data = {}
for tax, columns in dict_col_prediction.items():
    pred_data[tax] = francetax.select(*basic_columns, *columns).with_columns([
    create_insee_code(commune_code['DEPARTEMENT'], commune_code['COMMUNE']).alias("code INSEE")
]).collect()

In [13]:
for tax, data in pred_data.items():
    # Identify columns with Utf8 dtype
    utf8_columns = [col for col in data.columns if data[col].dtype == pl.Utf8]
    utf8_columns.remove('Commune name')
    utf8_columns.remove('DEPARTEMENT')
    utf8_columns.remove('code INSEE')
    utf8_columns.remove('COMMUNE')
    # if tax == 'TEOM':
    #     utf8_columns.remove('Bénéficiaire de la TEOM (C, I, P ou S)')
    #     utf8_columns.remove('Libellé du syndicat TEOM')
    #     utf8_columns.remove('NUMERO SIREN DU SYNDICAT TEOM')
    # Apply transformations to all relevant columns
    try:
        pred_data[tax] = data.with_columns([
            pl.when(pl.col(col).is_in([".", "'", ","]))
            .then(None)
            .otherwise(
                pl.col(col)
                .str.replace(',', '.')
                
            ).alias(col).cast(pl.Float64)
            for col in utf8_columns
        ])
    except Exception as e:
        print(f"Error processing {tax}: {e}")


# ML models

## Predict same year

### Clear data

In [14]:
def find_null_columns(df: pl.DataFrame, threshold: float = 0.5) -> list:
    total_rows = df.height
    null_counts = df.null_count().unpivot()
    
    result = (
        null_counts
        .filter(pl.col('value') / total_rows > threshold)
        .select('variable')
        .to_series()
        .to_list()
    )
    
    return result

In [15]:
# Drop columns with completeness < 80%
for tax, data in pred_data.items():
    for i in find_null_columns(data, 0.8):
        data.drop_in_place(i)

In [16]:
tax_column_dict = dict(zip(tax_types, y_columns))
for tax, target_column in tax_column_dict.items():
    pred_data[tax] = pred_data[tax].filter(pl.col(target_column).is_not_null())
    
    pred_data[tax] = pred_data[tax].select(pl.all().exclude('Commune name',	'DEPARTEMENT', 'COMMUNE', 'code INSEE'))
    pred_data[tax] = pred_data[tax].drop_nulls()

In [20]:
for tax, target_column in tax_column_dict.items():
    # Step 1: Filter rows where the target column is not null
    pred_data[tax] = (
        pred_data[tax]
        .filter(pl.col(target_column).is_not_null())
        .select(pl.all().exclude('Commune name', 'DEPARTEMENT', 'COMMUNE', 'code INSEE'))
        .drop_nulls()
    )

    # Step 2 & 3: Calculate the percentage of zero values for each column
    total_rows = pred_data[tax].height
    zero_percentage = pred_data[tax].select([
        ((pl.col(col) == 0).sum() / total_rows).alias(col)
        for col in pred_data[tax].columns
    ])
    
    # print(tax, zero_percentage)
    # Step 4: Keep only columns where less than 95% of the values are zero
    columns_to_keep = zero_percentage.unpivot().filter(
        pl.col("value") < 0.95
    ).select("variable").to_series().to_list()

    # Step 5: Select only the columns to keep
    pred_data[tax] = pred_data[tax].select(columns_to_keep)
    

### Create model

#### Decision Tree

In [68]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Function to train and evaluate decision tree regressor
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

tree_models = {}

for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    tree_models[tax] = train_evaluate_model(X_data, y_data)

In [75]:
# Print results
for tax, (model, mae_test, r2_test, mae_val, r2_val) in tree_models.items():
    # print(tax)
    print(f'{mae_test:.2f}')
    # print(f'{r2_test*100:.2f}%')
    # print(f"\nResults for {tax}:")
    # print(f"Test set - MSE: {mae_test:.4f}, R2: {r2_test:.4f}")
    # print(f"Validation set - MSE: {mae_val:.4f}, R2: {r2_val:.4f}")
    

873.05
4.29
27838.20
77756.47
4225.08
690.51


In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Function to train and evaluate decision tree regressor
def train_evaluate_model(X, y):
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Create and train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model, mse, r2

tree_models = {}

for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    tree_models[tax] = train_evaluate_model(X_data, y_data)


In [33]:
tree_models

{'FNB': (DecisionTreeRegressor(), 793.4263077037441, 0.990036247839628),
 'TAFNB': (DecisionTreeRegressor(), 4.133356197777793, 0.993969738855649),
 'FB': (DecisionTreeRegressor(), 39646.460659566226, 0.8791926567616005),
 'TH': (DecisionTreeRegressor(), 99414.51001624255, 0.9959456901462556),
 'CFE': (DecisionTreeRegressor(), 2948.9011406844106, 0.9997594293266302),
 'CVAE': (DecisionTreeRegressor(), 1195.3781331391604, 0.9930968841320498)}

#### Linear regression

In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

lin_models = {}

for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    lin_models[tax] = train_evaluate_model(X_data, y_data)

In [67]:
# Print results
for tax, (model, mae_test, r2_test, mae_val, r2_val) in lin_models.items():
    # print(tax)
    # print(f'{r2_val:.2f}')
    print(f'{r2_val*100:.2f}%')
    # print(f"\nResults for {tax}:")
    # print(f"Test set - MSE: {mae_test:.4f}, R2: {r2_test:.4f}")
    # print(f"Validation set - MSE: {mae_val:.4f}, R2: {r2_val:.4f}")

97.62%
22.90%
95.49%
100.00%
92.42%
100.00%


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np


# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
        
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model, mse, r2

lin_models = {}

for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    lin_models[tax] = train_evaluate_model(X_data, y_data)


In [36]:
lin_models

{'FNB': (LinearRegression(), 2284.250287146987, 0.9707233150151118),
 'TAFNB': (LinearRegression(), 354.58604633532946, 0.30478243880946176),
 'FB': (LinearRegression(), 164493.1441822689, 0.9401209022988952),
 'TH': (LinearRegression(), 39947.84598218239, 0.9977983866899214),
 'CFE': (LinearRegression(), 24206.65204181026, 0.9851223450405411),
 'CVAE': (LinearRegression(), 22.62728475225067, 0.9999999404110201)}

#### Most important features

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import numpy as np

# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model, mse, r2, X_train, y_train

def get_most_important_features(model, X_train, y_train, feature_names):
    # Calculate permutation importance
    results = permutation_importance(model, X_train, y_train, scoring='neg_mean_absolute_error')
    
    # Get the importance scores
    importance_scores = results.importances_mean
    
    # Get the sorted indices of features by importance
    sorted_indices = np.argsort(importance_scores)[::-1]
    
    # Create a list of most important features
    most_important_features = [feature_names[i] for i in sorted_indices]
    
    return most_important_features

lin_models = {}
important_features = {}

for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    
    model, mse, r2, X_train, y_train = train_evaluate_model(X_data, y_data)
    lin_models[tax] = model
    
    # Get feature names
    feature_names = pred_data[tax].select(pl.all().exclude(target_column)).columns
    
    # Get the most important features
    important_features[tax] = get_most_important_features(model, X_train, y_train, feature_names)


#### Calculate correlation

In [77]:
def calculate_correlations(X_data: pl.DataFrame, y_data: pl.DataFrame) -> pl.DataFrame:
    # Ensure y_data has only one column
    if y_data.shape[1] != 1:
        raise ValueError("y_data should only have one column")
    
    # Get the name of the single column in y_data
    y_column_name = y_data.columns[0]
    
    # Create an empty list to store the correlation results
    correlations = []
    
    # Calculate correlation for each column in X_data with the y_data column
    for col in X_data.columns:
        correlation = pl.select(pl.corr(X_data[col], y_data[y_column_name]))
        correlations.append((col, *correlation[col]))
    
    # Convert the results to a DataFrame
    correlation_df = pl.DataFrame(correlations, schema=["Feature", "Correlation"])
    
    return correlation_df

In [78]:
corr_df = {}
for tax, target_column in tax_column_dict.items():
    X_data = pred_data[tax].select(pl.all().exclude(target_column))
    y_data = pred_data[tax].select(target_column)
    corr_df[tax] = calculate_correlations(X_data, y_data)

C:\Users\lazzz\AppData\Local\Temp\ipykernel_11932\1167597072.py:18: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  correlation_df = pl.DataFrame(correlations, schema=["Feature", "Correlation"])


#### Test without highly correlated columns

#### Decision Tree

In [102]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Function to train and evaluate decision tree regressor
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = DecisionTreeRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

tree_models = {}

for tax, target_column in tax_column_dict.items():
    exclude_cols = corr_df[tax].filter(((pl.col('Correlation') < 0.7) & (pl.col('Correlation') > 0.0)))['Feature'].to_list() + [target_column]
    X_data = pred_data[tax].select(pl.all().exclude(exclude_cols)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    tree_models[tax] = train_evaluate_model(X_data, y_data)


In [110]:
# Print results
for tax, (model, mae_test, r2_test, mae_val, r2_val) in tree_models.items():
    # print(tax)
    # print(f'{mae_val:.0f}')
    print(f'{r2_val*100:.2f}%')
    # print(f"\nResults for {tax}:")
    # print(f"Test set - MSE: {mae_test:.4f}, R2: {r2_test:.4f}")
    # print(f"Validation set - MSE: {mae_val:.4f}, R2: {r2_val:.4f}")

97.21%
3.95%
96.85%
99.14%
98.51%
99.08%


#### Linear regression

In [129]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

lin_models = {}

for tax, target_column in tax_column_dict.items():
    exclude_cols = corr_df[tax].filter(((pl.col('Correlation') < 0.7) & (pl.col('Correlation') > 0.0)))['Feature'].to_list() + [target_column]
    X_data = pred_data[tax].select(pl.all().exclude(exclude_cols)).to_numpy()
    y_data = pred_data[tax].select(target_column).to_numpy().ravel()
    lin_models[tax] = train_evaluate_model(X_data, y_data)


In [133]:
# Print results
for tax, (model, mae_test, r2_test, mae_val, r2_val) in lin_models.items():
    # print(tax)
    print(f'{mae_val:.0f}')
    # print(f'{r2_test*100:.2f}%')
    # print(f"\nResults for {tax}:")
    # print(f"Test set - MSE: {mae_test:.4f}, R2: {r2_test:.4f}")
    # print(f"Validation set - MSE: {mae_val:.4f}, R2: {r2_val:.4f}")

3339
459
164840
118659
21865
15


In [139]:
corr_df['TH'].sort('Correlation')

Feature,Correlation
str,f64
"""TH - TAUX INTERCOMMUNAL TAXE G…",-0.052224
"""TH - SYNDICATS ET ORG. ASSIMIL…",-0.006778
"""TH - QUOTITE DES ABATTEMENTS A…",0.010322
"""TH - QUOTITE DES ABATTEMENTS A…",0.011104
"""TH - QUOTITE DES ABATTEMENTS A…",0.015064
"""TH - INTERCOMMUNALITE / TAUX V…",0.017563
"""TH - INTERCOMMUNALITE / TAUX A…",0.023983
"""TH - TSE GRAND PARIS OU EPFL G…",0.02676
"""TH - TSE / TAUX NET""",0.02679


In [146]:
corr_df['CVAE'].filter(((pl.col('Correlation')<0.8) & (pl.col('Correlation')>0.0)))

Feature,Correlation
str,f64
"""Part de CVAE au profit du GFP""",0.613394
"""Part de CVAE due au profit du …",0.611458
"""Part de CVAE dégrevée au profi…",0.608014
"""Part de CVAE exonérée compensé…",0.021761
"""Part de CVAE au profit du dépa…",0.437538
"""Part de CVAE due au profit du …",0.435462
"""Part de CVAE dégrevée au profi…",0.435169
"""Part de CVAE exonérée compensé…",0.000104
"""Part de CVAE au profit de la r…",0.620198


In [144]:
lin_models

{'FNB': (LinearRegression(), 69660150.01043834, 0.954765801300399),
 'TAFNB': (LinearRegression(), 5818611.77202496, 0.019531322371426296),
 'FB': (LinearRegression(), 875282029282.2172, 0.9780925972698389),
 'TH': (LinearRegression(), 7452630410263.093, 0.951195560483361),
 'CFE': (LinearRegression(), 11716629769.860273, 0.9997405567589526),
 'CVAE': (LinearRegression(), 356555.6092389835, 0.9999999517788819)}

# Future years prediction model

## Get data

In [16]:
year_pred_data = {}
for tax, columns in dict_col_prediction.items():
    year_pred_data[tax] = francetax.select(*basic_columns, 'ANNEE', *columns).with_columns([
    create_insee_code(commune_code['DEPARTEMENT'], commune_code['COMMUNE']).alias("code INSEE")
]).collect()

In [17]:
def find_null_columns(df: pl.DataFrame, threshold: float = 0.5) -> list:
    total_rows = df.height
    null_counts = df.null_count().unpivot()
    
    result = (
        null_counts
        .filter(pl.col('value') / total_rows > threshold)
        .select('variable')
        .to_series()
        .to_list()
    )
    
    return result

In [18]:
# Drop columns with completeness < 80%
for tax, data in year_pred_data.items():
    for i in find_null_columns(data, 0.8):
        data.drop_in_place(i)

In [19]:
for tax, data in year_pred_data.items():
    # Identify columns with Utf8 dtype
    utf8_columns = [col for col in data.columns if data[col].dtype == pl.Utf8]
    utf8_columns.remove('Commune name')
    utf8_columns.remove('DEPARTEMENT')
    utf8_columns.remove('code INSEE')
    utf8_columns.remove('COMMUNE')
    # if tax == 'TEOM':
    #     utf8_columns.remove('Bénéficiaire de la TEOM (C, I, P ou S)')
    #     utf8_columns.remove('Libellé du syndicat TEOM')
    #     utf8_columns.remove('NUMERO SIREN DU SYNDICAT TEOM')
    # Apply transformations to all relevant columns
    try:
        year_pred_data[tax] = data.with_columns([
            pl.when(pl.col(col).is_in([".", "'", ","]))
            .then(None)
            .otherwise(
                pl.col(col)
                .str.replace(',', '.')
                
            ).alias(col).cast(pl.Float64)
            for col in utf8_columns
        ])
    except Exception as e:
        print(f"Error processing {tax}: {e}")


In [20]:
target_years = 3
for tax, target_column in zip(tax_types, y_columns):
    for i in range(1, target_years + 1):
        # Prepare the shifted year DataFrame
        shifted_df = year_pred_data[tax].select([
            pl.col('code INSEE'),
            (pl.col('ANNEE') - i).alias('ANNEE'),
            pl.col(target_column).alias(f'TARGET-{i}')
        ])
        
        # Perform the join operation
        year_pred_data[tax] = year_pred_data[tax].join(
            shifted_df,
            on=['code INSEE', 'ANNEE'],
            how='left'
        )

In [21]:
for tax in tax_types:
    # year_pred_data[tax] = year_pred_data[tax].select(pl.all().exclude('Commune name',	'DEPARTEMENT', 'COMMUNE', 'code INSEE'))
    year_pred_data[tax] = year_pred_data[tax].drop_nulls()

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import polars as pl

# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

year_lin_models = {}

for tax, data in year_pred_data.items():
    year_lin_models[tax] = []
    for i in range(1, target_years + 1):
        exclude_cols = [f'TARGET-{j}' for j in range(1, target_years + 1) if j != i] + [*basic_columns, 'code INSEE']
        df = data.select(pl.all().exclude(exclude_cols)).drop_nulls()
        X_data = df.select(pl.all().exclude('^TARGET.*$')).to_numpy()
        y_data = df.select(f'TARGET-{i}').to_numpy().ravel()
        year_lin_models[tax] += [train_evaluate_model(X_data, y_data)]




In [50]:
# Print results
for tax, results in year_lin_models.items():
    # print(f"\nResults for {tax}:")
    # print(tax)
    r_prt =''
    for i, (model, mae_test, r2_test, mae_val, r2_val) in enumerate(results, 1):
        r_prt += f'{r2_val*100:.2f} '
        
    print(f'{r_prt}')
        
        # print(f"TARGET-{i} - Test set - MAE: {mae_test:.4f}, R2: {r2_test:.4f}")
        # print(f"TARGET-{i} - Validation set - MAE: {mae_val:.4f}, R2: {r2_val:.4f}")

95.48 94.97 94.16 
74.74 53.48 47.39 
99.88 88.03 97.35 
99.91 98.34 99.61 
-2152386.62 75.39 88.18 
51.53 -12889.78 -31225.98 


In [51]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import polars as pl

# Function to train and evaluate decision tree regression
def train_evaluate_model(X, y):
    
    # First split: separate validation set
    X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.15)
    
    # Second split: separate train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2941)
    # 0.2941 ensures that test set is 25% of the total data (0.25 / 0.85 = 0.2941)
    
    # Create and train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    # Make predictions on test set
    y_pred_test = model.predict(X_test)
    
    # Evaluate the model on test set
    mae_test = mean_absolute_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Make predictions on validation set
    y_pred_val = model.predict(X_val)
    
    # Evaluate the model on validation set
    mae_val = mean_absolute_error(y_val, y_pred_val)
    r2_val = r2_score(y_val, y_pred_val)
    
    return model, mae_test, r2_test, mae_val, r2_val

year_tree_models = {}

for tax, data in year_pred_data.items():
    year_tree_models[tax] = []
    for i in range(1, target_years + 1):
        exclude_cols = [f'TARGET-{j}' for j in range(1, target_years + 1) if j != i] + [*basic_columns, 'code INSEE']
        df = data.select(pl.all().exclude(exclude_cols)).drop_nulls()
        X_data = df.select(pl.all().exclude('^TARGET.*$')).to_numpy()
        y_data = df.select(f'TARGET-{i}').to_numpy().ravel()
        year_tree_models[tax] += [train_evaluate_model(X_data, y_data)]

# Print results
for tax, results in year_tree_models.items():
    print(f"\nResults for {tax}:")
    for i, (model, mae_test, r2_test, mae_val, r2_val) in enumerate(results, 1):
        print(f"TARGET-{i} - Test set - MAE: {mae_test:.4f}, R2: {r2_test:.4f}")
        print(f"TARGET-{i} - Validation set - MAE: {mae_val:.4f}, R2: {r2_val:.4f}")



Results for FNB:
TARGET-1 - Test set - MAE: 470.7690, R2: 0.9757
TARGET-1 - Validation set - MAE: 470.1492, R2: 0.9666
TARGET-2 - Test set - MAE: 579.9401, R2: 0.9725
TARGET-2 - Validation set - MAE: 647.1596, R2: 0.9587
TARGET-3 - Test set - MAE: 727.8891, R2: 0.9602
TARGET-3 - Validation set - MAE: 731.3051, R2: 0.9619

Results for TAFNB:
TARGET-1 - Test set - MAE: 54.9145, R2: 0.7515
TARGET-1 - Validation set - MAE: 48.5281, R2: 0.8572
TARGET-2 - Test set - MAE: 66.7570, R2: 0.6538
TARGET-2 - Validation set - MAE: 63.1176, R2: 0.8168
TARGET-3 - Test set - MAE: 68.0825, R2: 0.5865
TARGET-3 - Validation set - MAE: 63.7053, R2: 0.7460

Results for FB:
TARGET-1 - Test set - MAE: 5424.0532, R2: 0.9996
TARGET-1 - Validation set - MAE: 5181.0302, R2: 0.9998
TARGET-2 - Test set - MAE: 11397.3207, R2: 0.9990
TARGET-2 - Validation set - MAE: 21206.5224, R2: 0.9872
TARGET-3 - Test set - MAE: 25576.2876, R2: 0.9204
TARGET-3 - Validation set - MAE: 15302.1511, R2: 0.9977

Results for TH:
TARGET

In [56]:
# Print results
for tax, results in year_tree_models.items():
    # print(f"\nResults for {tax}:")
    # print(tax)
    r_prt =''
    for i, (model, mae_test, r2_test, mae_val, r2_val) in enumerate(results, 1):
        r_prt += f'{r2_val*100:.2f}, '
        
    print(f'{r_prt}')
        

96.66, 95.87, 96.19, 
85.72, 81.68, 74.60, 
99.98, 98.72, 99.77, 
89.87, 99.50, 66.36, 
98.53, 87.87, 89.51, 
86.41, 8.24, 80.17, 


In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np


# Function to train and evaluate linear regression
def train_evaluate_model(X, y):
        
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model, mse, r2

year_lin_models = {}

for tax, data in year_pred_data.items():
    year_lin_models[tax] = []
    for i in range(1, target_years + 1):
        exclude_cols = [f'TARGET-{j}' for j in range(1, target_years + 1) if j != i] + [*basic_columns, 'code INSEE']
        df = data.select(pl.all().exclude(exclude_cols)).drop_nulls()
        X_data = df.select(pl.all().exclude('^TARGET.*$'))
        y_data = df.select(f'TARGET-{i}')
        year_lin_models[tax] += [train_evaluate_model(X_data, y_data)]


In [ ]:
year_lin_models

In [ ]:
calculate_correlations(year_pred_data['TH'], year_pred_data['TH'].select(f'TARGET-2')).filter(pl.col('Correlation').is_not_nan()).sort('Correlation', descending=True)